# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制的栈道穿越着一片郁郁葱葱的草地。栈道似乎是为了方便人们探索和享受自然而设计的，它直线延伸进入远方的景色中。草地两侧生长着繁茂的草和灌木，天空呈现出美丽的蓝色，配合着散布的白云，为整个场景增添了宁静和广阔的感觉。整个图像色彩鲜明，自然风光令人感到宁静和放松。这样的自然场景可能吸引了很多喜欢户外活动和自然摄影的人士。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图展示了一条木制的栈道穿越着一片郁郁葱葱的草地。栈道似乎是为了方便人们探索和享受自然而设计的，它直线延伸进入远方的景色中。草地两侧生长着繁茂的草和灌木，天空呈现出美丽的蓝色，配合着散布的白云，为整个场景增添了宁静和广阔的感觉。整个图像色彩鲜明，自然风光令人感到宁静和放松。这样的自然场景可能吸引了很多喜欢户外活动和自然摄影的人士。'

### 封装成一个函数 query_image_description

In [6]:
from openai import OpenAI
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [2]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

NameError: name 'query_image_description' is not defined

### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [8]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FED9B5CD40>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [19]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这是一张展示了人工智能领域中几种模型调整方法（tuning methods）的笔记。内容包括：

1. **Prompt Tuning** - 这种方法涉及使用较小的模型，并通过对输入数据X进行转换模式的调整，来优化模型性能。笔记里描述了输入数据被转换为其嵌入表示，并记为Y = WX，其中W是转换矩阵。

2. **Prefix Tuning** - 在这种调整方法中，在处理过程中使用了一个前缀（Prefix），该前缀通过一系列操作与原始输入数据结合，以优化性能。

3. **LoRA（Low-Rank Adaptation）** - 低秩适应技术针对模型权重W进行调整，在W上添加一个增量ΔW。ΔW由A和B矩阵的乘积计算得出，这种调整是为了在不显著增加参数数量的情况下，改善模型的特定部分。

此外，笔记本还提到了与LoRA相关的存储需求问题，如LAMA需要65GB而使用LoRA调整后只需78GB，而另一个变体QLoRA则降至48GB，显示了优化的节约性。

这些笔记显示了对不同调整技术的理解和应用，这对于在特定任务上优化模型的性能至关重要。


In [22]:
content = query_base64_image_description("images/gpt-4v.jpg", "解释下图里的文字内容、古诗出处以及文字的颜色？")
print(content)

这张图片显示的是用黑色墨水手写的一首古诗。诗的内容是：

白日依山尽，
黄河入海流。
欲穷千里目，
更上一层楼。

这首诗出自唐代诗人王之涣的《登鹳雀楼》，表达了登高望远时的豪情壮志和对广阔世界的向往。诗中描绘了一幅壮观的景象：夕阳西下，黄河奔流入海，作者希望通过更高的位置来拓宽视野，象征着追求更高远的人生理想。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [23]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是用黑色墨水手写的一首古诗。诗的内容是：

白日依山尽，
黄河入海流。
欲穷千里目，
更上一层楼。

这首诗出自唐代诗人王之涣的《登鹳雀楼》，表达了登高望远时的豪情壮志和对广阔世界的向往。诗中描绘了一幅壮观的景象：夕阳西下，黄河奔流入海，作者希望通过更高的位置来拓宽视野，象征着追求更高远的人生理想。

![](./images/handwriting_1.jpg)

In [25]:
content = query_base64_image_description("images/gpt-4v.jpg", "解释下图里的文字内容、古诗出处以及文字的颜色？")
display(Markdown(content))

图中的文字内容是一首古诗，内容为：

白日依山尽，
黄河入海流。
欲穷千里目，
更上一层楼。

这首诗出自宋代诗人王之涣的《登鹳雀楼》，表达了登高远望时开阔的心境和壮阔的景象。诗中通过对自然景色的描述，表达了诗人对未来的憧憬和寄托。

文字的颜色是黑色。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。